In [1]:
from sklearn import model_selection
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.ensemble import BaggingRegressor as BR
from sklearn.ensemble import ExtraTreesRegressor as ETR
import pandas as pd
import numpy as np
df = pd.read_excel('hackaton_data_5post.xlsx', index_col = 0) # Грузим датасет
info = df[['ADDRESS', 'CITY', 'REGION']] # Выделяем столбцы со справочной информацией в отдельный DataFrame
del df['ADDRESS'], df['CITY'], df['REGION'] # Удаляем из исходного датасета столбцы со справочной информацией
#Возможно, сами адреса магазинов учитывались бы при выборе наиболее доступных магазинов для клиентов магазина
df = df.fillna(0) # заменяем все NaN на 0
df['NEAR_Stations'].replace([u'остановка', u'автостанция', '0'], [1, 1, 0], inplace=True) # Остановка -> 1
df['Summ_daily'] = df['Postamat_daily'] + df['cashbox_daily'] # Создаём новый столбец с суммой выдач постаматов и кэшбоксов
dflearn = df.loc[df['Summ_daily'] != 0] # Выделяем те точки, где есть что-то
dfresult = df.loc[df['Summ_daily'] == 0] # Выделяем точки для предсказания
dflearn = dflearn.iloc[:,5:]
dfresult = dfresult.iloc[:,5:]
dflearn = shuffle(dflearn, random_state=25) # Перемешивает исходную выборку
dftarget = dflearn['Summ_daily']
del dflearn['Summ_daily']

In [2]:
n = len(dflearn)
deln = int(n*0.25)
dfl1 = dflearn[:deln]
dfl2 = dflearn[deln:2*deln]
dfl3 = dflearn[2*deln:3*deln]
dfl4 = dflearn[-deln:]
dft1 = dftarget[:deln]
dft2 = dftarget[deln:2*deln]
dft3 = dftarget[2*deln:3*deln]
dft4 = dftarget[-deln:]

In [3]:
dflerror = pd.concat([dfl2, dfl3, dfl4])
dfterror = pd.concat([dft2, dft3, dft4])
dflerror2 = dfl1
dfterror2 = dft1
model = BR(base_estimator = ETR(n_estimators = 21), n_estimators = 34)
model.fit(dflerror, dfterror)
arr = (model.predict(dflerror2) - dfterror2) / dfterror2
indexes = arr[abs(arr) >  2].index.tolist()
dflerror2 = dflerror2.drop(indexes)
dfterror2 = dfterror2.drop(indexes)
DFL1 = dflerror2
DFT1 = dfterror2

In [4]:
dflerror = pd.concat([dfl1, dfl3, dfl4])
dfterror = pd.concat([dft1, dft3, dft4])
dflerror2 = dfl2
dfterror2 = dft2
model = BR(base_estimator = ETR(n_estimators = 21), n_estimators = 34)
model.fit(dflerror, dfterror)
arr = (model.predict(dflerror2) - dfterror2) / dfterror2
indexes = arr[abs(arr) >  2].index.tolist()
dflerror2 = dflerror2.drop(indexes)
dfterror2 = dfterror2.drop(indexes)
DFL2 = dflerror2
DFT2 = dfterror2

In [5]:
dflerror = pd.concat([dfl1, dfl2, dfl4])
dfterror = pd.concat([dft1, dft2, dft4])
dflerror2 = dfl3
dfterror2 = dft3
model = BR(base_estimator = ETR(n_estimators = 21), n_estimators = 34)
model.fit(dflerror, dfterror)
arr = (model.predict(dflerror2) - dfterror2) / dfterror2
indexes = arr[abs(arr) >  2].index.tolist()
dflerror2 = dflerror2.drop(indexes)
dfterror2 = dfterror2.drop(indexes)
DFL3 = dflerror2
DFT3 = dfterror2

In [6]:
dflerror = pd.concat([dfl1, dfl2, dfl3])
dfterror = pd.concat([dft1, dft2, dft3])
dflerror2 = dfl4
dfterror2 = dft4
model = BR(base_estimator = ETR(n_estimators = 21), n_estimators = 34)
model.fit(dflerror, dfterror)
arr = (model.predict(dflerror2) - dfterror2) / dfterror2
indexes = arr[abs(arr) >  2].index.tolist()
dflerror2 = dflerror2.drop(indexes)
dfterror2 = dfterror2.drop(indexes)
DFL4 = dflerror2
DFT4 = dfterror2

In [7]:
DFL = pd.concat([DFL1, DFL2, DFL3, DFL4])
DFT = pd.concat([DFT1, DFT2, DFT3, DFT4])
DF = DFL.join(DFT)

In [8]:
model = BR(base_estimator = ETR(n_estimators = 21), n_estimators = 34)
model.fit(DFL, DFT) # Обучаем на всей выборке (то бишь там, где есть или постомат, или кэшбокс, или то и то)
predict_= model.predict(dfresult) # предикт на всю таблицу

In [9]:
dfresult['result'] = np.around(predict_, 2)
del dfresult['Summ_daily']

In [10]:
dfSum = df.loc[(df['Postamat_daily'] != 0) & (df['cashbox_daily'] != 0)]
my_mean = dfSum['Summ_daily'].mean()
#Выбор порогового значения для постоматов. Возможно поменять условие с учетом "успешности" постомата или, 
#Например, порог может выбирать ЛПР, или же устанавливаться исходя из результатов задач логистики
dfPost = dfresult.loc[dfresult['result'] > my_mean] 
dfCashbox = dfresult.loc[(dfresult['result'] < my_mean) & (dfresult['result'] > 1)]

##### Результаты
Таблица с постоматами и с кэшбоксами (предсказанные значения лежат в столбце result)

In [11]:
dfPost

,hh_500,value1,value2,POPULATION,NEAR_Malls,NEAR_Business_centers,NEAR_metro_rjd,NEAR_Stations,macro_salary_avg_yearly,result
1,6369.0,10068,3236,324698,1,1,0.0,1,423321,4.66
5,5023.0,10100,3484,482873,1,0,0.0,1,387679,5.14
12,6686.0,10094,3187,482873,1,0,0.0,1,387679,4.76
19,3773.0,10094,3114,324698,0,1,0.0,1,423321,4.43
20,5082.0,10042,3003,482873,1,1,0.0,1,387679,5.36
25,6469.0,10061,3010,482873,1,0,0.0,0,387679,5.06
33,3971.0,10094,3096,482873,1,0,0.0,0,387679,6.00
34,3148.0,10053,3083,324698,1,0,0.0,0,423321,4.75
43,5223.0,9987,2939,482873,1,0,0.0,1,387679,4.20
63,1072.0,10036,3090,482873,1,0,0.0,1,387679,4.51


In [12]:
dfCashbox

,hh_500,value1,value2,POPULATION,NEAR_Malls,NEAR_Business_centers,NEAR_metro_rjd,NEAR_Stations,macro_salary_avg_yearly,result
16,7339.0,10085,3142,482873,0,0,0.0,1,387679,3.05
17,7552.0,10036,2945,324698,0,1,0.0,0,423321,2.82
21,4982.0,10087,3159,104739,0,1,0.0,1,423321,2.51
22,7836.0,10054,3075,104739,0,0,0.0,1,423321,2.94
42,2022.0,10094,3139,11288,0,0,1.0,0,423321,1.55
48,1748.0,10063,2968,324698,0,0,1.0,1,423321,2.82
75,6036.0,9973,2829,482873,0,0,0.0,0,387679,2.46
76,6122.0,9984,2767,482873,0,0,0.0,0,387679,2.93
82,354.0,10100,3238,7117,0,0,0.0,0,387679,1.13
83,2071.0,10099,3187,9171,0,0,0.0,1,387679,1.19
